# Test Model performance 

## Identity Model

In [1]:
import torch
import os
import datetime
import pandas as pd
from  torch.utils.data import Dataset
import torchvision.transforms as transforms
from PIL import Image
import torch.nn as nn
import json
import argparse
import torch

parser = argparse.ArgumentParser()
parser.add_argument('-d','--device', default='cuda', type=str,
                    help='test device cuda or cpu(default: cuda)')
parser.add_argument('-m', '--model', default='resnet/resnet18.pth', type=str,
                    help='path of test model (default: resnet/resnet18.pth)')
parser.add_argument('-q', '--quantization', action='store_true',  help='disables CUDA training')
args = parser.parse_args()

# from torchsummary import summary
starttime = datetime.datetime.now()
device = torch.device(args.device)
if True == args.quantization :
    model = torch.jit.load(args.model).to(device)
else :
    model = torch.load(args.model).to(device)


# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# model = torch.load('xception.pth').to(device)
# model = torch.load('densenet201.pth').to(device)
# model = torch.load('vgg11_pruning_fc.pth').to(device)

# model = torch.load('vgg/vgg11.pth').to(device)
# model = torch.load('vgg/vgg11_bn.pth').to(device)
# model = torch.load('vgg/vgg13.pth').to(device)
# model = torch.load('vgg/vgg13_bn.pth').to(device)
# model = torch.load('vgg/vgg16.pth').to(device)
# model = torch.load('vgg/vgg16_bn.pth').to(device)
# model = torch.load('vgg/vgg19.pth').to(device)
# model = torch.load('vgg/vgg19_bn.pth').to(device)

# model = torch.load('resnet/resnet18_pruning.pth').to(device)
# model = torch.load('resnet/resnet18.pth').to(device)
# model = torch.load('resnet/resnet34.pth').to(device)
# model = torch.load('resnet/resnet50.pth').to(device)
# model = torch.load('resnet/resnet101.pth').to(device)
# model = torch.load('resnet/resnet152.pth').to(device)
# model = torch.load('inception_v3/inception_v3.pth').to(device)


# model = torch.jit.load('resnet/quantization/resnet18.pth').to(device)
# model = torch.jit.load('resnet/quantization/resnext101_32x8d.pth').to(device)
# model = torch.jit.load('quantization_models/mobilenet_v2.pth').to(device)
# model = torch.jit.load('quantization_models/inception_v3.pth').to(device)


# model_name = 'inception_v3'
# model = models.quantization.__dict__[model_name](pretrained=True,progress=True,quantize=True).to(device)

criterion = nn.CrossEntropyLoss()
print(device)
model.eval()

usage: ipykernel_launcher.py [-h] [-d DEVICE] [-m MODEL] [-q]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\bawanag\AppData\Roaming\jupyter\runtime\kernel-3aeabc46-111f-4c69-9a84-e86e402fd2ce.json


SystemExit: 2

In [4]:
root_dir = "../../my_imagenet/ImageNet_validset/"#identify the classification 
class_label_location = "../../my_imagenet/scrip/imagenet_class_index.txt"
class_names = []
with open(class_label_location) as f:
    lines=f.readlines()
    class_to_lable_json = lines[0]
    class_to_lable_dict = json.loads(s=class_to_lable_json)
    for key in class_to_lable_dict:
        class_names.append(class_to_lable_dict[key][0])
        
def get_class_from_path(value):
    for key in class_to_lable_dict:
        if value == class_to_lable_dict[key][0]:
            return int(key)

NameError: name 'json' is not defined

In [3]:
def get_meta(root_dir, dirs):
    """ Fetches the meta data for all the images and assigns labels.
    """
    paths, classes = [], []
    for i, dir_ in enumerate(dirs):
        for entry in os.scandir(root_dir + dir_):
            if (entry.is_file()):
                paths.append(entry.path)
                classes.append(get_class_from_path(dir_))
                
    return paths, classes

In [4]:
# Benign images we will assign class 0, and malignant as 1
paths, classes = get_meta(root_dir, class_names)

data = {
    'path': paths,
    'class': classes
}

data_df = pd.DataFrame(data, columns=['path', 'class'])
data_df = data_df.sample(frac=1).reset_index(drop=True) # Shuffles the data

In [5]:
batch_size = 32
log_time = 100
validset_size = len(data_df)
print(len(data_df))
print(data_df.head())

25777
                                                path  class
0  my_imagenet/ImageNet_validset/n03535780/ILSVRC...    602
1  my_imagenet/ImageNet_validset/n03720891/ILSVRC...    641
2  my_imagenet/ImageNet_validset/n02977058/ILSVRC...    480
3  my_imagenet/ImageNet_validset/n02088364/ILSVRC...    162
4  my_imagenet/ImageNet_validset/n02002556/ILSVRC...    127


In [6]:
class ImageNet10(Dataset):

    def __init__(self, df, transform=None):

        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        # Load image from path and get label
        x = Image.open(self.df['path'][index])
        try:
          x = x.convert('RGB') # To deal with some grayscale images in the data
        except:
          pass
        y = torch.tensor(int(self.df['class'][index]))

        if self.transform:
            x = self.transform(x)

        return x, y

In [7]:
data_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

ins_dataset_valid = ImageNet10(
#     df=data_df[0:int(len(data_df)/10)],
    df=data_df,
    transform=data_transform,
)
valid_dataset_loader = torch.utils.data.DataLoader(
    ins_dataset_valid,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2
)


In [8]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [9]:
def single_valid_dataset_gpu():
    print('=====================================================================')
    running_loss = 0.0
    correct = 0
    total = 0
    acc1 = []
    acc5 = []
    with torch.no_grad():  
        for i, data in enumerate(valid_dataset_loader, 0):
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            if i == 0 :
                starttime_each_log = datetime.datetime.now()
                print('Model inference time: {}'.format(starttime_each_log - starttime))
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            #             _, predicted = torch.max(outputs.data, 1)
            #             total += labels.size(0)
            #             total += len(labels)
            #             correct += (predicted == labels).sum().item()
            acc1_temp, acc5_temp = accuracy(outputs, labels, topk=(1, 5))

            acc1.append(acc1_temp)
            acc5.append(acc5_temp)
            if i%log_time == 0 and i != 0:
                endtime_each_log = datetime.datetime.now()
                time_comsumption = endtime_each_log - starttime_each_log
                starttime_each_log = endtime_each_log
                print('validset    Loss: {:.4f}   Top1 Accuracy : {:.4f}%  Top5 accuracy : {:.4f}%  Time comsumption:{}'.format(running_loss/(i*batch_size), acc1_temp.item(), acc5_temp.item(), time_comsumption)) 
    print("===========================================================================")
    acc1_sum = 0
    for acc1_temp in acc1:
        acc1_sum += acc1_temp
    acc1_average = acc1_sum/len(acc1)
    acc5_sum = 0
    for acc5_temp in acc5:
        acc5_sum += acc5_temp
    acc5_average = acc5_sum/len(acc5)
    print('overall validset    Loss: {:.4f}   Top1 Accuracy : {:.4f}%  Top5 accuracy : {:.4f}% Time comsumption:{}'.format(running_loss/(i*batch_size), acc1_average.item(), acc5_average.item(), (datetime.datetime.now() - starttime)))

## Testing

In [10]:
single_valid_dataset_gpu()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 128, 128]           9,408
       BatchNorm2d-2         [-1, 64, 128, 128]             128
              ReLU-3         [-1, 64, 128, 128]               0
         MaxPool2d-4           [-1, 64, 64, 64]               0
            Conv2d-5           [-1, 64, 64, 64]          36,864
       BatchNorm2d-6           [-1, 64, 64, 64]             128
              ReLU-7           [-1, 64, 64, 64]               0
            Conv2d-8           [-1, 64, 64, 64]          36,864
       BatchNorm2d-9           [-1, 64, 64, 64]             128
             ReLU-10           [-1, 64, 64, 64]               0
       BasicBlock-11           [-1, 64, 64, 64]               0
           Conv2d-12           [-1, 64, 64, 64]          36,864
      BatchNorm2d-13           [-1, 64, 64, 64]             128
             ReLU-14           [-1, 64,